In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 决策树解决分类问题

In [ ]:
def DecisionTreeClassifierTest(X, y, criterion, max_depth=2):
    def plot_decision_boundary(model, axis):
        """绘制决策边界"""
        x0, x1 = np.meshgrid(
            np.linspace(axis[0], axis[1], int((axis[1]-axis[0])*100)),
            np.linspace(axis[2], axis[3], int((axis[3]-axis[2])*100))
        )
        X_new = np.c_[x0.ravel(), x1.ravel()]

        y_predict = model.predict(X_new)
        zz = y_predict.reshape(x0.shape)

        from matplotlib.colors import ListedColormap
        custom_cmap = ListedColormap(['#EF9A9A', '#FFF59D', '#90CAF9'])

        plt.contourf(x0, x1, zz, cmap=custom_cmap)


    def plot_scatter(X, y):
        plt.scatter(X[y == 0, 0], X[y == 0, 1])
        plt.scatter(X[y == 1, 0], X[y == 1, 1])
        plt.scatter(X[y == 2, 0], X[y == 2, 1])
    
    from sklearn.tree import DecisionTreeClassifier
    dt_clf = DecisionTreeClassifier(
        max_depth=max_depth, criterion=criterion, random_state=66)
    dt_clf.fit(X, y)
    plot_decision_boundary(dt_clf, axis=[0.5, 7.5, 0, 3])
    plot_scatter(X, y)

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data[:, 2:]
y = iris.target

## 信息熵
- **公式:** H(X)= −∑{p(*xi*)`*`log[p(*xi*)]}, *i*∈[1, m]，p(*xi*)表示类别*xi*发生的概率，所以∑p(*xi*)=1
- **实际意义:** 熵越大，数据的不确定性越高。当 p(*xk*)=1 时，H(X)=0，表示数据归为一类，不确定性最低

### 划分效果

In [ ]:
DecisionTreeClassifierTest(X, y, criterion='entropy')

### 函数直观理解

In [ ]:
def plot_entropy():
    """二分类问题的信息熵"""
    def entropy(p): 
        return -p * np.log(p) - (1 - p) * np.log(1 - p)
    
    x = np.linspace(0.01, 0.99, 200)
    plt.plot(x, entropy(x))
    plt.show()

In [ ]:
plot_entropy()

### 模拟寻找最优划分

In [ ]:
def entropy(y):
    """统计数据类别，计算信息熵"""
    from collections import Counter
    from math import log
    counter = Counter(y)   # 统计类别
    res = 0.0
    for num in counter.values():
        p = num / len(y)    # 计算每个类别的概率
        res += -p * log(p)  # 计算熵
    return res


def split(X, y, d, v):
    """根据某一个维度d和某一个阈值v，将数据集划分为两部分"""
    index_a = (X[:, d] <= v)
    index_b = (X[:, d] > v)
    return X[index_a], X[index_b], y[index_a], y[index_b]


def try_split(X, y, f_wave):
    """迭代找到划分维度d、划分阈值v"""
    best_wave = np.float32('inf')
    best_d, best_v = -1, -1

    # 遍历每个维度
    for d in range(X.shape[1]):
        sorted_index = np.argsort(X[:, d])
        # 遍历本维度所有数据
        for i in range(1, len(X)):
            if X[sorted_index[i-1], d] == X[sorted_index[i], d]:
                continue

            # 1.取两个相邻的数据，以中间值为阈值，尝试划分为两部分
            v = (X[sorted_index[i-1], d] + X[sorted_index[i], d]) / 2
            X_l, X_r, y_l, y_r = split(X, y, d, v)

            # 2.计算本次划分后的信息熵或基尼系数
            e = f_wave(y_l) + f_wave(y_r)

            # 3.判断指标是否最小
            if e < best_wave:
                best_wave, best_d, best_v = e, d, v

    return best_wave, best_d, best_v


def plot_split(X, y, best_d, best_v, color='r'):
    plt.scatter(X[y == 0, 0], X[y == 0, 1])
    plt.scatter(X[y == 1, 0], X[y == 1, 1])
    plt.scatter(X[y == 2, 0], X[y == 2, 1])

    min_ = np.min(X[:, 1-best_d])
    max_ = np.max(X[:, 1-best_d])
    if best_d == 0:
        plt.plot([best_v, best_v], [min_, max_], color=color)
    else:
        plt.plot([min_, max_], [best_v, best_v], color=color)

**第一次划分**

In [ ]:
# 节点一: 迭代划分维度d、阈值v
best_entropy, best_d, best_v = try_split(X, y, f_wave=entropy)
plot_split(X, y, best_d, best_v, color='r')
print("信息熵:", best_entropy)
print("维度  :", best_d)
print("阈值  :", best_v)

In [ ]:
# 节点一: 在维度d上，根据阈值v划分后的两部分
X1_l, X1_r, y1_l, y1_r = split(X, y, best_d , best_v)

In [ ]:
entropy(y1_l)  # 左半部分已被完全划分

In [ ]:
entropy(y1_r)  # 右半部分还可以继续划分

**第二次划分**

In [ ]:
# 节点二: 迭代划分维度d、阈值v
best_entropy2, best_d2, best_v2 = try_split(X1_r, y1_r, f_wave=entropy)
plot_split(X, y, best_d2, best_v2, color='b')
print("信息熵:", best_entropy2)
print("维度  :", best_d2)
print("阈值  :", best_v2)

In [ ]:
# 节点二: 在维度d上，根据阈值v划分后的两部分
X2_l, X2_r, y2_l, y2_r = split(X1_r, y1_r, best_d2 , best_v2)

In [ ]:
entropy(y2_l)

In [ ]:
entropy(y2_r)

## 基尼系数
- **公式:** G(X)= 1 − ∑{p(*xi*)^2}, *i*∈[1, m]，p(*xi*)表示类别*xi*发生的概率，所以∑p(*xi*)=1
- **实际意义:** 基尼系数越大，数据的不确定性越高。当 p(*xk*)=1 时，H(X)=0，表示数据归为一类，不确定性最低

### 划分效果

In [ ]:
DecisionTreeClassifierTest(X, y, criterion='gini')

### 函数直观理解

In [ ]:
def plot_gini():
    """二分类问题的基尼系数"""
    def gini(p): 
        return 1 - (p ** 2 + (1 - p) ** 2)
    
    x = np.linspace(0.01, 0.99, 200)
    plt.plot(x, gini(x))
    plt.show()

In [ ]:
plot_gini()

### 模拟寻找最优划分

In [ ]:
def gini(y):
    """统计数据类别，计算信息熵"""
    from collections import Counter
    from math import log
    counter = Counter(y)   # 统计类别
    res = 1.0
    for num in counter.values():
        p = num / len(y)    # 计算每个类别的概率
        res -= p ** 2       # 计算基尼系数
    return res

**第一次划分**

In [ ]:
# 节点一: 迭代划分维度d、阈值v
best_gini, best_d, best_v = try_split(X, y, f_wave=gini)
plot_split(X, y, best_d, best_v, color='r')
print("基尼系数:", best_gini)
print("维度  :", best_d)
print("阈值  :", best_v)

In [ ]:
# 节点一: 在维度d上，根据阈值v划分后的两部分
X1_l, X1_r, y1_l, y1_r = split(X, y, best_d , best_v)

In [ ]:
gini(y1_l)  # 左半部分已被完全划分

In [ ]:
gini(y1_r)  # 右半部分还可以继续划分

**第二次划分**

In [ ]:
# 节点二: 迭代划分维度d、阈值v
best_gini2, best_d2, best_v2 = try_split(X1_r, y1_r, f_wave=gini)
plot_split(X, y, best_d2, best_v2, color='b')
print("基尼系数:", best_gini2)
print("维度  :", best_d2)
print("阈值  :", best_v2)

In [ ]:
gini(y2_l)

In [ ]:
gini(y2_r)

# 决策树解决回归问题

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def plot_learning_curve(algo, X_train, X_test, y_train, y_test):
    """绘制学习曲线"""
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.metrics import mean_squared_error

    train_score = []
    test_score = []
    for i in range(1, len(X_train)+1):
        algo.fit(X_train[:i], y_train[:i])

        y_train_predict = algo.predict(X_train[:i])
        train_score.append(mean_squared_error(y_train[:i], y_train_predict))

        y_test_predict = algo.predict(X_test)
        test_score.append(mean_squared_error(y_test, y_test_predict))

    plt.plot([i for i in range(1, len(X_train)+1)], np.sqrt(train_score), label='train')
    plt.plot([i for i in range(1, len(X_train)+1)], np.sqrt(test_score), label='test')
    plt.legend()
    plt.show()

In [ ]:
# 1.加载数据集
boston = load_boston()
X = iris.data[:, 2:]
y = iris.target

# 2.划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)

# 3.模型训练
def DecisionTreeRegressorTest(model):
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    plot_learning_curve(model, X_train, X_test, y_train, y_test)

In [ ]:
DecisionTreeRegressorTest(DecisionTreeRegressor())

In [ ]:
DecisionTreeRegressorTest(DecisionTreeRegressor(max_depth=1))

In [ ]:
DecisionTreeRegressorTest(DecisionTreeRegressor(max_depth=2))

In [ ]:
DecisionTreeRegressorTest(DecisionTreeRegressor(max_depth=3))

In [ ]:
dt_reg = DecisionTreeRegressor(max_depth=2)
dt_reg.fit(X_train, y_train)
y_predict = dt_reg.predict(X_test)

plt.scatter(np.arange(len(X_test)), y_test, color='g')
plt.scatter(np.arange(len(X_test)), y_predict, color='r')
plt.show()